In [1]:
import pandas as pd
import numpy as np
from mcbac import CifHelper
import mcbac

# Using pymatgen

In [2]:
import pymatgen
from pymatgen.io.cif import CifParser

In [3]:
# load cif and get dict
p = CifParser('./Example.cif')
d = p.as_dict()

In [4]:
# make helper object
c = CifHelper(d['NiH16C24N2O5'])

In [5]:
c.nebr_stack_charge()

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
4,H,C,CHN,0.124004,0.102895,0.086959,0.086959,-0.001120,0.088079
...,...,...,...,...,...,...,...,...,...
187,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
188,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
189,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
190,O,CC,CCCCOO,-0.641530,-0.255959,-0.235748,-0.235748,-0.003035,-0.232713


# from gemmi

In [6]:
import gemmi.cif
import json

In [7]:
# note that the File Example.cif has a weird header
# this fixes that
with open('./Example.cif','r') as f:
    out = f.readlines()

with open('./Example.gemmi.cif', 'w') as f:
    f.writelines(out[1:])

In [8]:
# or using gemmi
g = gemmi.cif.read_file('./Example.gemmi.cif')

d_gemmi = json.loads(g.as_json(lowercase_names=False))
c2 = CifHelper(d_gemmi['NiH16C24N2O5'])

In [9]:
pd.testing.assert_frame_equal(c.nebr_stack(), c2.nebr_stack())

# from methods

In [11]:
dict_of_objects_0 = CifHelper.from_pymatgen('./Example.cif', sole_block=False )

dc0 = dict_of_objects_0['NiH16C24N2O5']

dict_of_objects_1 = CifHelper.from_gemmi('./Example.gemmi.cif', sole_block=False)

dc1 = dict_of_objects_1['nih16c24n2o5']

In [12]:
pd.testing.assert_frame_equal(c.nebr_stack(), dc0.nebr_stack())
pd.testing.assert_frame_equal(c.nebr_stack(), dc1.nebr_stack())

In [13]:
# or just use sole_block=True, which is the default

In [14]:
dc0 = CifHelper.from_pymatgen('./Example.cif')
dc1 = CifHelper.from_gemmi('./Example.gemmi.cif',)

In [15]:
pd.testing.assert_frame_equal(c.nebr_stack(), dc0.nebr_stack())
pd.testing.assert_frame_equal(c.nebr_stack(), dc1.nebr_stack())

# Basic functionality

In [16]:
# directly access things from cif file
c['_cell_length_a']

'11.26900037'

In [17]:
# created renamers for easy access and float/array conversion 
# see class definition for more info
print(c.a)

11.26900037


In [18]:
help(c)

Help on CifHelper in module mcbac.core object:

class CifHelper(builtins.object)
 |  CifHelper(data, radii_dict=None, name=None)
 |  
 |  Helper class to work with cif file
 |  
 |  Paramters
 |  ---------
 |  data : dict
 |      dictionary from cif file
 |  radii_dict : dict, optional
 |      mapping from elements -> radii
 |  
 |  Attributes
 |  ----------
 |  a, b, c : float
 |      values from data['_cell_length_a'] etc
 |  alpha, beta, gamma : float
 |      value from data['_cell_angle_alpha], etc
 |  label : array of strings
 |      value of data['_atom_site_label']
 |  type_sybol : array of strings
 |      value from data['_atom_site_type_symbol']
 |      or data['_atom_site_label] if type_symbol not available.
 |  frac_x, frac_y, frac_z, charge : array of floats
 |      value from data['_atom_site_fract_x'], etc
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, index)
 |  
 |  __init__(self, data, radii_dict=None, name=None)
 |      Initialize self.  See help(type(self)

## cartisian coords

In [19]:
# coordinates -> c.coords
# replicated coords -> c.coords_replicated(stack=?)

## distance calc

In [20]:
# unstacked has shape (n, 27, n)
# first dim is for coords, second for replicate id, third replicate atom id
# if stacked, then shape = (n, 27 * n)
c.distance_matrix(stack=True)

array([[31.49257799, 27.47389044, 21.95667209, ..., 41.29246609,
        46.47352217, 40.24557049],
       [36.51919306, 31.49257799, 23.96030471, ..., 36.38409112,
        41.58302591, 34.37499822],
       [46.03417839, 40.40185054, 31.49257799, ..., 28.56033942,
        33.75433917, 25.25947674],
       ...,
       [41.29246609, 36.38409112, 28.56033942, ..., 31.49257799,
        36.91356271, 29.89882206],
       [46.47352217, 41.58302591, 33.75433917, ..., 26.21199769,
        31.49257799, 24.65199383],
       [40.24557049, 34.37499822, 25.25947674, ..., 34.83530307,
        40.03525782, 31.49257799]])

## distance dataframe

In [21]:
c.distance_frame

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance
0,0,0,0,1.164,1.164,Ni,Ni,31.492578
1,0,0,1,1.164,1.164,Ni,Ni,27.473890
2,0,0,2,1.164,1.164,Ni,Ni,21.956672
3,0,0,3,1.164,1.164,Ni,Ni,23.960305
4,0,0,4,1.164,0.354,Ni,H,16.968970
...,...,...,...,...,...,...,...,...
995323,191,26,187,0.634,0.634,O,O,31.157935
995324,191,26,188,0.634,0.634,O,O,35.151915
995325,191,26,189,0.634,0.634,O,O,34.835303
995326,191,26,190,0.634,0.634,O,O,40.035258


## nearest neighbor frame

In [22]:
c.nebr_frame(rcut_fac=1.25)

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank
0,0,13,0,1.164,1.164,Ni,Ni,0.000000e+00,2.91000,0
1,0,13,174,1.164,0.634,Ni,O,2.008001e+00,2.24750,1
2,0,12,185,1.164,0.634,Ni,O,2.036361e+00,2.24750,2
3,0,13,165,1.164,0.700,Ni,N,2.087743e+00,2.33000,3
4,0,13,164,1.164,0.700,Ni,N,2.098025e+00,2.33000,4
...,...,...,...,...,...,...,...,...,...,...
627,190,13,140,0.634,0.757,O,C,1.385512e+00,1.73875,1
628,190,13,143,0.634,0.757,O,C,1.390848e+00,1.73875,2
629,191,13,191,0.634,0.634,O,O,8.881784e-16,1.58500,0
630,191,13,157,0.634,0.757,O,C,1.386235e+00,1.73875,1


## nebr of nebr frame

In [23]:
# second neighbors of 'cntr' are in 'nebr_2nd'
c.nebr_nebr_frame(rcut_fac=1.25)

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank,cntr_2nd,repl_2nd,nebr_2nd,cntr_radii_2nd,nebr_radii_2nd,cntr_name_2nd,nebr_name_2nd,distance_2nd,rcut_2nd,rank_2nd
0,0,13,174,1.164,0.634,Ni,O,2.008001,2.24750,1,174,13,102,0.634,0.757,O,C,1.245089,1.73875,1
1,0,13,174,1.164,0.634,Ni,O,2.008001,2.24750,1,174,13,0,0.634,1.164,O,Ni,2.008001,2.24750,2
2,0,12,185,1.164,0.634,Ni,O,2.036361,2.24750,2,185,13,150,0.634,0.757,O,C,1.261929,1.73875,1
3,0,12,185,1.164,0.634,Ni,O,2.036361,2.24750,2,185,14,0,0.634,1.164,O,Ni,2.036361,2.24750,2
4,0,13,165,1.164,0.700,Ni,N,2.087743,2.33000,3,165,13,75,0.700,0.757,N,C,1.324517,1.82125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,191,13,157,0.634,0.757,O,C,1.386235,1.73875,1,157,13,158,0.757,0.757,C,C,1.384673,1.89250,2
1220,191,13,157,0.634,0.757,O,C,1.386235,1.73875,1,157,13,191,0.757,0.634,C,O,1.386235,1.73875,3
1221,191,13,154,0.634,0.757,O,C,1.395728,1.73875,2,154,4,155,0.757,0.757,C,C,1.358932,1.89250,1
1222,191,13,154,0.634,0.757,O,C,1.395728,1.73875,2,154,13,153,0.757,0.757,C,C,1.364052,1.89250,2


## 0th, 1st, and 2nd neighbor keys

In [24]:
c.nebr_stack()

,nebr_0,nebr_1,nebr_2
cntr,,,
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi
4,H,C,CHN
...,...,...,...
187,O,CNi,CNNOOOOOO
188,O,CNi,CNNOOOOOO
189,O,CNi,CNNOOOOOO


In [25]:
# test this is the same as from other code
other = pd.read_csv('./Example.cif.dat.total.txt', header=None, sep='\s+').values
np.testing.assert_array_equal(c.nebr_stack().values, other)

# Thats where I'm at.  Not sure what to do with the databases

I don't understand what the shell script is trying to do.  The first read merge I get, but the other ones make no sense.

In [23]:
# OK, I now realize what the awk code is doing
# First tries to use triples, then doubles, then singles (2nd, 1st, 0th)

In [26]:
out = c.nebr_stack_charge()

In [27]:
out

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
1,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
2,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
3,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,-0.010461,0.822937
4,H,C,CHN,0.124004,0.102895,0.086959,0.086959,-0.001120,0.088079
...,...,...,...,...,...,...,...,...,...
187,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
188,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
189,O,CNi,CNNOOOOOO,-0.641530,-0.564211,-0.570475,-0.570475,-0.007345,-0.563130
190,O,CC,CCCCOO,-0.641530,-0.255959,-0.235748,-0.235748,-0.003035,-0.232713


In [28]:
# test against awk values...
other_total = pd.read_csv('./Final_Charge_Example.cif.txt',header=None)[0].values
other_shift = pd.read_csv('./Shift_Charge_Example.cif.txt',header=None)[0].values
other_zero = pd.read_csv('./Final_Charge_Zeroed_Example.cif.txt', header=None)[0].values

np.testing.assert_allclose(out['val_total'], other_total)
np.testing.assert_allclose(out['val_zero'], other_zero, rtol=1e-6, atol=1e-5)
np.testing.assert_allclose(out['val_shift'], other_shift, rtol=1e-6, atol=1e-5)

# TODO:

I don't know how to write back the cif file.  Played with some stuff, but failed.  Calling it there for now...

In [45]:
c = CifHelper.from_pymatgen('../origsrc/New_11600_CBAC/FINAL_ABEYAJ_clean.cif')

In [46]:
np.testing.assert_allclose(c.nebr_stack_charge()['val_zero'], c.charge, atol=1e-5, rtol=1e-4)

AssertionError: 
Not equal to tolerance rtol=0.0001, atol=1e-05

Mismatched elements: 120 / 144 (83.3%)
Max absolute difference: 0.00157801
Max relative difference: 0.00729341
 x: array([ 2.233269,  2.207979,  2.233269,  2.207979,  2.233269,  2.207979,
        2.233269,  2.207979,  0.083456,  0.115347,  0.083456,  0.115347,
        0.083456,  0.115347,  0.083456,  0.115347,  0.083456,  0.115347,...
 y: array([ 2.23359 ,  2.2083  ,  2.23359 ,  2.2083  ,  2.23359 ,  2.2083  ,
        2.23359 ,  2.2083  ,  0.083524,  0.115349,  0.083524,  0.115349,
        0.083524,  0.115349,  0.083524,  0.115349,  0.083524,  0.115349,...

In [35]:
c.charge

array([ 0.969325 ,  0.969325 ,  0.969325 ,  0.969325 ,  0.0893798,
        0.115748 ,  0.115748 ,  0.0893798,  0.0893798,  0.115748 ,
        0.115748 ,  0.0893798,  0.0893798,  0.115748 ,  0.115748 ,
        0.0893798,  0.0893798,  0.115748 ,  0.115748 ,  0.0893798,
        0.0893798,  0.115748 ,  0.115748 ,  0.0893798,  0.0893798,
        0.115748 ,  0.115748 ,  0.0893798,  0.0893798,  0.115748 ,
        0.115748 ,  0.0893798,  0.0893798,  0.115748 ,  0.115748 ,
        0.0893798,  0.17419  , -0.179435 , -0.131413 , -0.179435 ,
        0.17419  ,  0.701519 ,  0.17419  , -0.179435 , -0.131413 ,
       -0.179435 ,  0.17419  ,  0.701519 ,  0.17419  , -0.179435 ,
       -0.131413 , -0.179435 ,  0.17419  ,  0.701519 ,  0.17419  ,
       -0.179435 , -0.131413 , -0.179435 ,  0.17419  ,  0.701519 ,
        0.17419  , -0.179435 , -0.131413 , -0.179435 ,  0.17419  ,
        0.701519 ,  0.17419  , -0.179435 , -0.131413 , -0.179435 ,
        0.17419  ,  0.701519 ,  0.17419  , -0.179435 , -0.1314

In [29]:
from importlib import reload
import mcbac.core as core
reload(core)

<module 'mcbac.core' from '/Users/wpk/Documents/python/projects/mcbac/mcbac/core.py'>

In [173]:
c = core.CifHelper.from_pymatgen('../tests/test_data/magnetochemistry3010001_PF290NdNO3_clean.cif')

In [167]:
c.distance_frame.query('(cntr != nebr and central_image) or (not central_image)').query('cntr==0').sort_values(['cntr','distance'])

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,central_image
108174,0,10,14,1.816,0.634,Nd,O,2.360417,False
43272,0,4,8,1.816,0.634,Nd,O,2.424271,False
43279,0,4,15,1.816,0.634,Nd,O,2.491457,False
140614,0,13,6,1.816,0.634,Nd,O,2.498909,True
140615,0,13,7,1.816,0.634,Nd,O,2.503181,True
...,...,...,...,...,...,...,...,...,...
281305,0,26,89,1.816,0.634,Nd,O,35.528398,False
37,0,0,37,1.816,0.634,Nd,O,35.595710,False
66,0,0,66,1.816,0.634,Nd,O,35.840701,False
22,0,0,22,1.816,0.634,Nd,O,37.335594,False


In [157]:
c.nebr_stack()

,nebr_0,nebr_1,nebr_2
cntr,,,
0,Nd,OOOOOOO,CCCCCCNNdNdNdNdNdNdNd
1,Nd,OOOOOOO,CCCCCCCNdNdNdNdNdNdNdNdNd
2,O,CNd,CNdOOOOOOOOO
3,O,CNdNd,CNdOOOOOOOOOOOOOOOO
4,O,CNd,COOOOOOOOO
...,...,...,...
99,H,C,CHN
100,C,HNN,CCCCCCC
101,H,C,HNN


In [119]:
(
    c.distance_frame
    .assign(rcut=lambda x: (x['cntr_radii'] + x['nebr_radii']) * 1.25)
    .query('distance < rcut')
    .sort_values(['cntr','distance'])
    .groupby('cntr')
    
)

In [146]:
center_mapping = pd.DataFrame({'central_image': (c.offsets == 0).all(-1), 'repl': np.arange(len(c.offsets))})

In [148]:
c.nebr_frame(nnebr=None).query('cntr==0').merge(center_mapping)

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank,central_image
0,0,13,0,1.816,1.816,Nd,Nd,0.000000,4.5400,0,True
1,0,13,6,1.816,0.634,Nd,O,2.498909,3.0625,4,True
2,0,13,7,1.816,0.634,Nd,O,2.503181,3.0625,5,True
3,0,13,19,1.816,0.634,Nd,O,2.514380,3.0625,6,True
4,0,13,26,1.816,0.634,Nd,O,2.594832,3.0625,8,True
5,0,13,12,1.816,0.700,Nd,N,2.998322,3.1450,9,True
6,0,10,14,1.816,0.634,Nd,O,2.360417,3.0625,1,False
7,0,4,8,1.816,0.634,Nd,O,2.424271,3.0625,2,False
8,0,4,15,1.816,0.634,Nd,O,2.491457,3.0625,3,False
9,0,14,22,1.816,0.634,Nd,O,2.594497,3.0625,7,False


In [139]:
o

,cntr,repl,nebr,cntr_radii,nebr_radii,cntr_name,nebr_name,distance,rcut,rank,central_image
0,0,13,0,1.816,1.816,Nd,Nd,0.000000,4.5400,0,13
1,0,10,14,1.816,0.634,Nd,O,2.360417,3.0625,1,10
2,0,4,8,1.816,0.634,Nd,O,2.424271,3.0625,2,4
3,0,4,15,1.816,0.634,Nd,O,2.491457,3.0625,3,4
4,0,13,6,1.816,0.634,Nd,O,2.498909,3.0625,4,13
5,0,13,7,1.816,0.634,Nd,O,2.503181,3.0625,5,13
6,0,13,19,1.816,0.634,Nd,O,2.514380,3.0625,6,13
7,0,14,22,1.816,0.634,Nd,O,2.594497,3.0625,7,14
8,0,13,26,1.816,0.634,Nd,O,2.594832,3.0625,8,13
9,0,13,12,1.816,0.700,Nd,N,2.998322,3.1450,9,13


In [135]:
center_mapping[o['repl']]

13     True
10    False
4     False
4     False
13     True
13     True
13     True
14    False
13     True
13     True
dtype: bool

In [110]:
t = pd.read_csv('../awk_stuff/magnetochemistry3010001_PF290NdNO3_clean.cif.dat.total.txt', sep='\s+', header=None).values

In [111]:
a = c.nebr_stack().values

In [112]:
np.testing.assert_array_equal(a, t)

In [104]:
t

array([['Nd', 'OOOOOOO', 'CCCCCCNNdNdNdNdNdNdNd'],
       ['Nd', 'OOOOOOO', 'CCCCCCCNdNdNdNdNdNdNdNdNd'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['O', 'CNdNd', 'CNdOOOOOOOOOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['C', 'CHHN', 'CCCCCCOO'],
       ['H', 'C', 'CHHN'],
       ['H', 'C', 'CHHN'],
       ['N', 'NdOOO', 'NNNNdNdOOOOOOO'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['C', 'CNdOO', 'CCCHHNNdNdNdOOOOOOO'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'NNd', 'NdOOOOOOOOOO'],
       ['C', 'COO', 'CCCHHNNdNd'],
       ['C', 'COO', 'CCCHHNNdNd'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'NNd', 'NdOOOOOOOOOO']

In [81]:
t

array([['Nd', 'OOOOOOO', 'CCCCCCNNdNdNdNdNdNdNd'],
       ['Nd', 'OOOOOOO', 'CCCCCCCNdNdNdNdNdNdNdNdNd'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['O', 'CNdNd', 'CNdOOOOOOOOOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['C', 'CHHN', 'CCCCCCOO'],
       ['H', 'C', 'CHHN'],
       ['H', 'C', 'CHHN'],
       ['N', 'NdOOO', 'NNNNdNdOOOOOOO'],
       ['O', 'CNd', 'CNdOOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'CNd', 'COOOOOOOOO'],
       ['C', 'CNdOO', 'CCCHHNNdNdNdOOOOOOO'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'NNd', 'NdOOOOOOOOOO'],
       ['C', 'COO', 'CCCHHNNdNd'],
       ['C', 'COO', 'CCCHHNNdNd'],
       ['N', 'CCC', 'CCHHHHNNNN'],
       ['O', 'NNd', 'NdOOOOOOOOOO']

# Other crap

In [53]:
from pathlib import Path

In [71]:
paths= [(x.name,) for x in Path('../awk_stuff/').glob('*.cif') if not x.name.startswith('FINAL_')]

In [72]:
paths

[('magnetochemistry3010001_PF290NdNO3_clean.cif',),
 ('Example.cif',),
 ('jp102463p_si_002_clean.cif',),
 ('jacs.6b08724_ja6b08724_si_016_clean.cif',)]

In [62]:
paths[0].name.startswith('FINAL_')

True

In [33]:
cc = list(CifHelper.from_pymatgen('../awk_stuff/jacs.6b08724_ja6b08724_si_016_clean.cif').values())[0]

In [52]:
cc.nebr_stack_charge().assign(y=cc.charge)

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero,y
0,O,CNi,COOOOOOO,-0.641530,-0.564211,-0.588865,-0.588865,-0.014777,-0.574088,-0.574048
1,C,CCH,CCCCCCH,0.104865,-0.105721,-0.062860,-0.062860,-0.001577,-0.061283,-0.061279
2,H,C,CCH,0.124004,0.102895,0.112627,0.112627,-0.002826,0.115453,0.115447
3,C,CCH,CCCCCCH,0.104865,-0.105721,-0.062860,-0.062860,-0.001577,-0.061283,-0.061279
4,H,C,CCH,0.124004,0.102895,0.112627,0.112627,-0.002826,0.115453,0.115447
...,...,...,...,...,...,...,...,...,...,...
258,N,CCCo,CCCCNNNNNN,-0.297278,-0.353531,NaN,-0.353531,-0.008871,-0.344659,-0.344635
259,N,CCCo,CCCCNNNNNN,-0.297278,-0.353531,NaN,-0.353531,-0.008871,-0.344659,-0.344635
260,N,CCCo,CCCCNNNNNN,-0.297278,-0.353531,NaN,-0.353531,-0.008871,-0.344659,-0.344635
261,N,CCCo,CCCCNNNNNN,-0.297278,-0.353531,NaN,-0.353531,-0.008871,-0.344659,-0.344635


In [29]:
# toy example

In [30]:
c.nebr_stack().head().to_dict(orient='records')

[{'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'CCCCCCCCNiNiNiNiNiNi'},
 {'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'CCCCCCCCNiNiNiNiNiNi'},
 {'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'CCCCCCCCNiNiNiNiNiNi'},
 {'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'CCCCCCCCNiNiNiNiNiNi'},
 {'nebr_0': 'H', 'nebr_1': 'C', 'nebr_2': 'CHN'}]

In [31]:
tmp = pd.DataFrame([
    {'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'CCCCCCCCNiNiNiNiNiNi'},
    {'nebr_0': 'Ni', 'nebr_1': 'NNOOOO', 'nebr_2': 'ZZ'},
    {'nebr_0': 'Ni', 'nebr_1': 'ZZ', 'nebr_2': 'ZZ'},
    {'nebr_0': 'Z', 'nebr_1':'ZZ', 'nebr_2':'ZZ'},
    {'nebr_0': 'Z', 'nebr_1':'ZZ', 'nebr_2':'ZZ'},

])


In [32]:
c.nebr_stack_charge(nebr_stack=tmp)

,nebr_0,nebr_1,nebr_2,val_0,val_1,val_2,val_total,val_shift,val_zero
0,Ni,NNOOOO,CCCCCCCCNiNiNiNiNiNi,0.702856,0.840692,0.812476,0.812476,0.0,0.812476
1,Ni,NNOOOO,ZZ,0.702856,0.840692,NaN,0.840692,0.0,0.840692
2,Ni,ZZ,ZZ,0.702856,NaN,NaN,0.702856,0.0,0.702856
3,Z,ZZ,ZZ,NaN,NaN,NaN,-1.178012,0.0,-1.178012
4,Z,ZZ,ZZ,NaN,NaN,NaN,-1.178012,0.0,-1.178012


# Crap playing around with writing back to cif.  Failing

In [328]:
c.distance_frame

cntr_radii  nebr_radii cntr_name nebr_name   distance
cntr repl nebr                                                       
0    0    0          1.164       1.164        Ni        Ni  31.492578
          1          1.164       1.164        Ni        Ni  27.473890
          2          1.164       1.164        Ni        Ni  21.956672
          3          1.164       1.164        Ni        Ni  23.960305
          4          1.164       0.354        Ni         H  16.968970
...                    ...         ...       ...       ...        ...
191  26   187        0.634       0.634         O         O  31.157935
          188        0.634       0.634         O         O  35.151915
          189        0.634       0.634         O         O  34.835303
          190        0.634       0.634         O         O  40.035258
          191        0.634       0.634         O         O  31.492578

[995328 rows x 5 columns]

In [329]:
dist = c.distance_matrix(stack=False)

In [331]:
dist.shape

(192, 27, 192)

In [332]:
cntr, repl, nebr = np.mgrid[0:len(c), 0:27, 0:len(c)]

In [368]:
%%timeit -n 3 -r 3
df = pd.DataFrame({
    'cntr': cntr.flatten(),
    'repl': repl.flatten(),
    'nebr': nebr.flatten(),
    'cntr_radii': c.atom_radii[cntr].flatten(),
    'nebr_radii': c.atom_radii[nebr].flatten(),
    'cntr_name': c.type_symbol[cntr].flatten(),
    'nebr_name': c.type_symbol[nebr].flatten(),
    'distance': dist.flatten(),
}).set_index(['cntr','repl','nebr'])

73.3 ms ± 3.56 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [ ]:
pd.testing.assert_frame_equal(df, c.distance_frame[df.columns])

In [381]:
#%%timeit -n 3 -r 3
df2 = (
    pd.DataFrame({
    'cntr': cntr.flatten(),
    'repl': repl.flatten(),
    'nebr': nebr.flatten(),
    'distance': dist.flatten()})
    .assign(
        cntr_radii=lambda x: c.atom_radii[x['cntr']],
        nebr_radii=lambda x: c.atom_radii[x['nebr']],
        cntr_name=lambda x: c.type_symbol[x['cntr']],
        nebr_name=lambda x: c.type_symbol[x['nebr']]
    )
    .set_index(['cntr','repl','nebr'])
)


In [382]:
pd.testing.assert_frame_equal(df2, c.distance_frame[df2.columns])

In [388]:
cntr, repl, nebr = np.mgrid[0:len(c), 0:27, 0:len(c)].reshape(3, -1)


df3 = pd.DataFrame({
    'cntr': cntr,
    'repl': repl,
    'nebr': nebr,
    'cntr_radii': c.atom_radii[cntr],
    'nebr_radii': c.atom_radii[nebr],
    'cntr_name': c.type_symbol[cntr],
    'nebr_name': c.type_symbol[nebr],
    'distance': dist.flatten(),
})#.set_index(['cntr','repl','nebr'])

In [390]:
pd.testing.assert_frame_equal(df3, c.distance_frame)

In [ ]:
## pymatgen = not the way to go

In [33]:
import pymatgen.io.cif as cif

In [34]:
pp = cif.CifParser('./Example.cif')

In [288]:
s = pp.get_structures()

w = cif.CifWriter(s[0])

w.write_file('out_pymatgen.cif')

array([0.53469, 0.36462, 0.93074])

In [44]:
pp2 = cif.CifParser('./out1.cif')

In [46]:
s2 = pp2.get_structures()[0]

In [56]:
print(str(w))

# generated using pymatgen
data_NiH16C24N2O5
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   11.27000012
_cell_length_b   11.26900037
_cell_length_c   20.71100000
_cell_angle_alpha   77.60300104
_cell_angle_beta   77.60800134
_cell_angle_gamma   64.72699811
_symmetry_Int_Tables_number   1
_chemical_formula_structural   NiH16C24N2O5
_chemical_formula_sum   'Ni4 H64 C96 N8 O20'
_cell_volume   2300.48801702
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Ni  Ni0  1  0.53469000  0.36462000  0.93074000  1
  Ni  Ni1  1  0.63547000  0.46530000  0.56919000  1
  Ni  Ni2  1  0.46531000  0.63538000  0.06926000  1
  Ni  Ni3  1  0.36453000  0.53470000  0.43081000  1
  H  H4  1  0.31490000  0.30270000  0.02290000  1
  H  H5  1  0.11350000  0.30130000  0.02360000  1
  H  H6 

In [57]:
import gemmi.cif
import json

In [289]:
# or using gemmi
doc = gemmi.cif.read_file('./Example.gemmi.cif')

In [291]:
doc.check_for_missing_values()

In [290]:
gemmi.cif.read_mmjson('./out_pymatgen.json')

RuntimeError: 

In [242]:
doc.write_file('tmp.cif')

In [ ]:
doc

In [303]:
block = doc.sole_block()

In [304]:
table = block.find('_atom_site_', ['fract_x','fract_y','fract_z'])

In [252]:
for t in table:
    break

In [259]:
for item in block:
    print(item.pair)


['_symmetry_space_group_name_H-M', "'P 1'"]
['_cell_length_a', '11.26900037']
['_cell_length_b', '11.27000012']
['_cell_length_c', '20.71100000']
['_cell_angle_alpha', '77.60800134']
['_cell_angle_beta', '77.60300104']
['_cell_angle_gamma', '64.72699811']
['_symmetry_Int_Tables_number', '1']
['_chemical_formula_structural', 'NiH16C24N2O5']
['_chemical_formula_sum', "'Ni4 H64 C96 N8 O20'"]
['_cell_volume', '2300.4880165']
['_cell_formula_units_Z', '4']
None
None


In [258]:
item.pair

In [256]:
t[2]

'0.06926'

In [260]:
fract_x = block.find_values('loop_')

In [278]:
with open('tmp.json','w') as f:
    f.write(doc.as_json(mmjson=True))

In [279]:
gemmi.cif.read_mmjson('./tmp.json')

RuntimeError: 

In [284]:
%psource gemmi.cif

No source found for gemmi.cif


In [282]:
gemmi.cif.read_mmjson('./Example.gemmi.json')

RuntimeError: 

In [280]:
gemmi.cif.read_mmjson('./HEM.json')

<gemmi.cif.Document with 1 blocks (HEM)>

In [266]:
print(doc.as_string())

data_NiH16C24N2O5
_symmetry_space_group_name_H-M 'P 1'
_cell_length_a 11.26900037
_cell_length_b 11.27000012
_cell_length_c 20.71100000
_cell_angle_alpha 77.60800134
_cell_angle_beta 77.60300104
_cell_angle_gamma 64.72699811
_symmetry_Int_Tables_number 1
_chemical_formula_structural NiH16C24N2O5
_chemical_formula_sum 'Ni4 H64 C96 N8 O20'
_cell_volume 2300.4880165
_cell_formula_units_Z 4

loop_
_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1 'x, y, z'

loop_
_atom_site_label
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
_atom_site_charge
Ni 0.63538 0.46531 0.06926 0.840593
Ni 0.5347 0.36453 0.43081 0.837707
Ni 0.36462 0.53469 0.93074 0.840593
Ni 0.4653 0.63547 0.56919 0.837707
H 0.6973 0.6851 0.9771 0.074724
H 0.6987 0.8865 0.9764 0.118461
H 0.5687 0.8601 0.1703 0.123411
H 0.5697 0.6588 0.1654 0.090662
H 0.7804 0.023 0.9948 0.134363
H 0.7775 0.2258 0.9967 0.073416
H 0.5048 0.2849 0.1506 0.093039
H 0.5011 0.0823 0.1545 0.124105
H 0.3394 0.4311 0.3342 0.091925
H 0.1419

In [276]:
doc.write_file('tmp.json', )

In [247]:
for x in fract_x:
    print(x)

0.63538
0.5347
0.36462
0.4653
0.6973
0.6987
0.5687
0.5697
0.7804
0.7775
0.5048
0.5011
0.3394
0.1419
0.1145
0.315
0.7146
0.9167
0.7748
0.9772
0.8503
0.0364
0.2589
0.0759
0.2747
0.3547
0.4372
0.3626
0.1131
0.9065
0.0602
0.2673
0.8155
0.7241
0.6817
0.7632
0.3027
0.3013
0.4313
0.4303
0.2196
0.2225
0.4952
0.4989
0.6606
0.8581
0.8855
0.685
0.2854
0.0833
0.2252
0.0228
0.1497
0.9636
0.7411
0.9241
0.7253
0.6453
0.5628
0.6374
0.8869
0.0935
0.9398
0.7327
0.1845
0.2759
0.3183
0.2368
0.6699
0.6716
0.6338
0.5952
0.5959
0.6369
0.7222
0.7195
0.5603
0.5571
0.2934
0.1744
0.1015
0.1584
0.2792
0.7679
0.8895
0.9718
0.803
0.9254
0.8266
0.9439
0.9327
0.0439
0.1649
0.177
0.0671
0.3121
0.3089
0.3567
0.4073
0.4063
0.3606
0.4617
0.3473
0.2116
0.1034
0.9791
0.9658
0.0714
0.1948
0.7977
0.787
0.7339
0.6949
0.7072
0.7573
0.6374
0.3301
0.3284
0.3662
0.4048
0.4041
0.3631
0.2778
0.2805
0.4397
0.4429
0.7066
0.8256
0.8985
0.8416
0.7208
0.2321
0.1105
0.0282
0.197
0.0746
0.1734
0.0561
0.0673
0.9561
0.8351
0.823
0.9329
0.68

In [114]:
# convert to json
d_gemmi = json.loads(doc.as_json(lowercase_names=False))

In [226]:
d_gemmi

{'NiH16C24N2O5': {'_symmetry_space_group_name_H-M': 'P 1',
  '_cell_length_a': 11.26900037,
  '_cell_length_b': 11.27000012,
  '_cell_length_c': 20.711,
  '_cell_angle_alpha': 77.60800134,
  '_cell_angle_beta': 77.60300104,
  '_cell_angle_gamma': 64.72699811,
  '_symmetry_Int_Tables_number': 1,
  '_chemical_formula_structural': 'NiH16C24N2O5',
  '_chemical_formula_sum': 'Ni4 H64 C96 N8 O20',
  '_cell_volume': 2300.4880165,
  '_cell_formula_units_Z': 4,
  '_symmetry_equiv_pos_site_id': [1],
  '_symmetry_equiv_pos_as_xyz': ['x, y, z'],
  '_atom_site_label': ['Ni',
   'Ni',
   'Ni',
   'Ni',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   '

In [116]:
dd = d_gemmi['NiH16C24N2O5']

In [169]:
block = doc.sole_block()

In [ ]:
doc.sole_block

In [188]:
l = block.find_values('_atom_site_fract_x')

In [191]:
s = l.get_loop()

In [292]:
import CifFile

In [299]:
out = CifFile.ReadCif('./Example.gemmi.cif')

In [302]:
CifFile.CifFile_module.CifFile.

CifFile.CifFile_module.CifFile

In [300]:
x = out[out.keys()[0]]

In [297]:
x['_atom_site_charge'][0] = -1

In [298]:
out.co

In [232]:
with open('tt.cif','w') as f:
    f.write(out.WriteOut())

Writing NiH16C24N2O5, <CifFile.StarFile.StarBlock object at 0x7fe3507b17c0>
All blocks output.


In [233]:
cc = CifHelper.from_pymatgen('tt.cif')

In [238]:
cc['NiH16C24N2O5']['']

{'NiH16C24N2O5': <mcbac.core.CifHelper at 0x7fe362c2fb50>}

In [237]:
pd.testing.assert_frame_equal(cc['NiH16C24N2O5'].nebr_stack_charge(), c.nebr_stack_charge())

In [227]:
out.dictionary['nih16c24n2o5']

In [193]:
s.values

['Ni',
 '0.63538',
 '0.46531',
 '0.06926',
 '0.840593',
 'Ni',
 '0.5347',
 '0.36453',
 '0.43081',
 '0.837707',
 'Ni',
 '0.36462',
 '0.53469',
 '0.93074',
 '0.840593',
 'Ni',
 '0.4653',
 '0.63547',
 '0.56919',
 '0.837707',
 'H',
 '0.6973',
 '0.6851',
 '0.9771',
 '0.074724',
 'H',
 '0.6987',
 '0.8865',
 '0.9764',
 '0.118461',
 'H',
 '0.5687',
 '0.8601',
 '0.1703',
 '0.123411',
 'H',
 '0.5697',
 '0.6588',
 '0.1654',
 '0.090662',
 'H',
 '0.7804',
 '0.023',
 '0.9948',
 '0.134363',
 'H',
 '0.7775',
 '0.2258',
 '0.9967',
 '0.073416',
 'H',
 '0.5048',
 '0.2849',
 '0.1506',
 '0.093039',
 'H',
 '0.5011',
 '0.0823',
 '0.1545',
 '0.124105',
 'H',
 '0.3394',
 '0.4311',
 '0.3342',
 '0.091925',
 'H',
 '0.1419',
 '0.4289',
 '0.3294',
 '0.123513',
 'H',
 '0.1145',
 '0.3012',
 '0.5238',
 '0.118414',
 'H',
 '0.315',
 '0.3039',
 '0.5228',
 '0.074001',
 'H',
 '0.7146',
 '0.4944',
 '0.3496',
 '0.09416',
 'H',
 '0.9167',
 '0.4988',
 '0.3454',
 '0.122487',
 'H',
 '0.7748',
 '0.2226',
 '0.5039',
 '0.074742',
 

In [170]:
for element in block:
    print(element.pair)

['_symmetry_space_group_name_H-M', "'P 1'"]
['_cell_length_a', '11.26900037']
['_cell_length_b', '11.27000012']
['_cell_length_c', '20.71100000']
['_cell_angle_alpha', '77.60800134']
['_cell_angle_beta', '77.60300104']
['_cell_angle_gamma', '64.72699811']
['_symmetry_Int_Tables_number', '1']
['_chemical_formula_structural', 'NiH16C24N2O5']
['_chemical_formula_sum', "'Ni4 H64 C96 N8 O20'"]
['_cell_volume', '2300.4880165']
['_cell_formula_units_Z', '4']
None
None


In [135]:
element = block[0]

In [167]:
element.pair

['_symmetry_space_group_name_H-M', "'P 1'"]

In [153]:
e = block.find_values('_atom_site_label')

In [156]:
l = e.get_loop()

TypeError: get_loop(): incompatible function arguments. The following argument types are supported:
    1. (self: gemmi.cif.Column) -> gemmi.cif.Loop

Invoked with: <gemmi.cif.Column _atom_site_label length 192>, '_atom_site_label'

['Ni',
 '0.63538',
 '0.46531',
 '0.06926',
 '0.840593',
 'Ni',
 '0.5347',
 '0.36453',
 '0.43081',
 '0.837707',
 'Ni',
 '0.36462',
 '0.53469',
 '0.93074',
 '0.840593',
 'Ni',
 '0.4653',
 '0.63547',
 '0.56919',
 '0.837707',
 'H',
 '0.6973',
 '0.6851',
 '0.9771',
 '0.074724',
 'H',
 '0.6987',
 '0.8865',
 '0.9764',
 '0.118461',
 'H',
 '0.5687',
 '0.8601',
 '0.1703',
 '0.123411',
 'H',
 '0.5697',
 '0.6588',
 '0.1654',
 '0.090662',
 'H',
 '0.7804',
 '0.023',
 '0.9948',
 '0.134363',
 'H',
 '0.7775',
 '0.2258',
 '0.9967',
 '0.073416',
 'H',
 '0.5048',
 '0.2849',
 '0.1506',
 '0.093039',
 'H',
 '0.5011',
 '0.0823',
 '0.1545',
 '0.124105',
 'H',
 '0.3394',
 '0.4311',
 '0.3342',
 '0.091925',
 'H',
 '0.1419',
 '0.4289',
 '0.3294',
 '0.123513',
 'H',
 '0.1145',
 '0.3012',
 '0.5238',
 '0.118414',
 'H',
 '0.315',
 '0.3039',
 '0.5228',
 '0.074001',
 'H',
 '0.7146',
 '0.4944',
 '0.3496',
 '0.09416',
 'H',
 '0.9167',
 '0.4988',
 '0.3454',
 '0.122487',
 'H',
 '0.7748',
 '0.2226',
 '0.5039',
 '0.074742',
 

In [130]:
gemmi.cif.Document

gemmi.cif.Document

In [96]:
data = json.loads(doc.as_json(mmjson=True))

In [98]:
data

{'data_NiH16C24N2O5': {'symmetry_space_group_name_H-M': ['P 1'],
  'cell_length_a': [11.26900037],
  'cell_length_b': [11.27000012],
  'cell_length_c': [20.711],
  'cell_angle_alpha': [77.60800134],
  'cell_angle_beta': [77.60300104],
  'cell_angle_gamma': [64.72699811],
  'symmetry_Int_Tables_number': [1],
  'chemical_formula_structural': ['NiH16C24N2O5'],
  'chemical_formula_sum': ['Ni4 H64 C96 N8 O20'],
  'cell_volume': [2300.4880165],
  'cell_formula_units_Z': [4],
  'symmetry_equiv_pos_site_id': [1],
  'symmetry_equiv_pos_as_xyz': ['x, y, z'],
  'atom_site_label': ['Ni',
   'Ni',
   'Ni',
   'Ni',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H',
   'H'

In [97]:
gemmi.cif.read_mmjson('one.json')

RuntimeError: 

In [79]:
block = doc.sole_block() 
for element in block.find_loop("_atom_site_fract_x"):
    if element not in greeted:
        print("Hello " + element)
        greeted.add(element)


Hello 0.63538
Hello 0.5347
Hello 0.36462
Hello 0.4653
Hello 0.6973
Hello 0.6987
Hello 0.5687
Hello 0.5697
Hello 0.7804
Hello 0.7775
Hello 0.5048
Hello 0.5011
Hello 0.3394
Hello 0.1419
Hello 0.1145
Hello 0.315
Hello 0.7146
Hello 0.9167
Hello 0.7748
Hello 0.9772
Hello 0.8503
Hello 0.0364
Hello 0.2589
Hello 0.0759
Hello 0.2747
Hello 0.3547
Hello 0.4372
Hello 0.3626
Hello 0.1131
Hello 0.9065
Hello 0.0602
Hello 0.2673
Hello 0.8155
Hello 0.7241
Hello 0.6817
Hello 0.7632
Hello 0.3027
Hello 0.3013
Hello 0.4313
Hello 0.4303
Hello 0.2196
Hello 0.2225
Hello 0.4952
Hello 0.4989
Hello 0.6606
Hello 0.8581
Hello 0.8855
Hello 0.685
Hello 0.2854
Hello 0.0833
Hello 0.2252
Hello 0.0228
Hello 0.1497
Hello 0.9636
Hello 0.7411
Hello 0.9241
Hello 0.7253
Hello 0.6453
Hello 0.5628
Hello 0.6374
Hello 0.8869
Hello 0.0935
Hello 0.9398
Hello 0.7327
Hello 0.1845
Hello 0.2759
Hello 0.3183
Hello 0.2368
Hello 0.6699
Hello 0.6716
Hello 0.6338
Hello 0.5952
Hello 0.5959
Hello 0.6369
Hello 0.7222
Hello 0.7195
Hello 0.5603

In [74]:
str(element)

'<gemmi.cif.Item object at 0x7fe350780d30>'

In [ ]:
c2 = CifHelper(d_gemmi['NiH16C24N2O5'])

In [ ]:
pd.testing.assert_frame_equal(c.nebr_stack(), c2.nebr_stack())

In [ ]:
gemmi.cif.read('./one.json')

In [ ]:
d_gemmi['NiH16C24N2O5'].keys()

In [ ]:
g.as_json(mmjson=True)

In [ ]:
with open('tmp.json', 'w') as f:
    f.write(g.as_json(lowercase_names=False))

In [ ]:
with open('tmp.json','r') as f:
    j = json.load(f)

In [ ]:
gammi

In [ ]:
import pymatgen.io.cif as cif

In [ ]:
p.get_bibtex_string()

In [ ]:
o = cif.CifWriter(ss)

In [ ]:
o.write_file('tmp_pymat.cif')

In [ ]:
ss.

In [ ]:
pymatgen.core.Structure.from_file('./Example.cif')